### Import dataset

In [29]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


data = pd.read_json('worldCup.json')
players = pd.read_json('player.json')

data = data[data.playerId !=0]

data.head()

,eventId,eventName,eventSec,id,matchId,matchPeriod,playerId,positions,subEventId,subEventName,tags,teamId
0,8,Pass,1.892339,263883958,2058017,1H,14943,"[{'y': 51, 'x': 50}]",85,Simple pass,[],Croatia
1,8,Pass,3.889375,263883959,2058017,1H,69968,"[{'y': 52, 'x': 39}, {'y': 74, 'x': 34}]",85,Simple pass,[{'id': 1801}],Croatia
2,8,Pass,6.140946,263883960,2058017,1H,8287,"[{'y': 74, 'x': 34}, {'y': 93, 'x': 30}]",85,Simple pass,[{'id': 1801}],Croatia
3,8,Pass,9.226570,263883963,2058017,1H,69409,"[{'y': 93, 'x': 30}, {'y': 65, 'x': 11}]",85,Simple pass,[{'id': 1801}],Croatia
4,8,Pass,12.658969,263883964,2058017,1H,135747,"[{'y': 65, 'x': 11}, {'y': 51, 'x': 64}]",84,Launch,[{'id': 1801}],Croatia


In [30]:
def make_list(list_dict, key='id'):
    if len(list_dict) == 0:
        return []
    return [i[key] for i in list_dict]

def get_x(lista):
    if not lista:
        return None
    return lista[0]['x']

def get_y(lista):
    if not lista:
        return None
    return lista[0]['y']

In [31]:
data.tags = data.tags.apply(make_list)
data['x'] = data.positions.apply(get_x)
data['y'] = data.positions.apply(get_y)

data.drop('positions', axis=1, inplace=True)

bins = np.arange(0,120,20)
label = [0,1,2,3,4]
data['numero_cella_x'] = pd.cut(data.x, bins = bins, labels = label).fillna(0.).astype(int)
data['numero_cella_y'] = pd.cut(data.y, bins = bins, labels = label).fillna(0.).astype(int)
data.head()

,eventId,eventName,eventSec,id,matchId,matchPeriod,playerId,subEventId,subEventName,tags,teamId,x,y,numero_cella_x,numero_cella_y
0,8,Pass,1.892339,263883958,2058017,1H,14943,85,Simple pass,[],Croatia,50,51,2,2
1,8,Pass,3.889375,263883959,2058017,1H,69968,85,Simple pass,[1801],Croatia,39,52,1,2
2,8,Pass,6.140946,263883960,2058017,1H,8287,85,Simple pass,[1801],Croatia,34,74,1,3
3,8,Pass,9.226570,263883963,2058017,1H,69409,85,Simple pass,[1801],Croatia,30,93,1,4
4,8,Pass,12.658969,263883964,2058017,1H,135747,84,Launch,[1801],Croatia,11,65,0,3


## Eventi totali 

In [32]:
total_events_by_team = data.groupby(['teamId']).size().reset_index(name = 'total_events_cell')
total_events_by_cell = data.groupby(['numero_cella_x','numero_cella_y','teamId']).size().reset_index(name = 'number_events_cell')

df = total_events_by_cell.merge(total_events_by_team, on = ['teamId'])
df['events_freq'] = df.number_events_cell/df.total_events_cell 
df.drop(['number_events_cell','total_events_cell'],1,inplace=True)

df.head()

,numero_cella_x,numero_cella_y,teamId,events_freq
0,0,0,Croatia,0.026713
1,0,1,Croatia,0.018583
2,0,2,Croatia,0.013937
3,0,3,Croatia,0.015099
4,0,4,Croatia,0.012776


## Passaggi

In [33]:
accurate_pass = data.loc[[idx for idx,row in data.iterrows() if 1801 in row['tags'] and row.eventId == 8]]

accurate_pass_by_team = accurate_pass.groupby(['teamId']).size().reset_index(name = 'num_total_pass')
accurate_pass_by_cell = accurate_pass.groupby(['numero_cella_x','numero_cella_y','teamId']).size().reset_index(name = 'number_acc_pass')

df_pass = accurate_pass_by_cell.merge(accurate_pass_by_team, on = ['teamId'])
df_pass['acc_pass_freq'] = df_pass.number_acc_pass/df_pass.num_total_pass

df = df.merge(df_pass[['numero_cella_x','numero_cella_y','teamId','acc_pass_freq']], how = 'left',on = ['numero_cella_x','numero_cella_y','teamId'])

## Tiri

In [34]:
shot = data.loc[data.eventId == 10]

shot_by_team = shot.groupby(['teamId']).size().reset_index(name = 'num_total_shot')
shot_by_cell = shot.groupby(['numero_cella_x','numero_cella_y','teamId']).size().reset_index(name = 'number_shot')

df_shot = shot_by_cell.merge(shot_by_team, on = ['teamId'])
df_shot['shot_freq'] = df_shot.number_shot/df_shot.num_total_shot

df = df.merge(df_shot[['numero_cella_x','numero_cella_y','teamId','shot_freq']], how = 'left',on = ['numero_cella_x','numero_cella_y','teamId'])

## Falli

In [35]:
foul = data.loc[(data.eventId == 2) & (data.subEventId ==20)]

foul_by_team = foul.groupby(['teamId']).size().reset_index(name = 'num_total_foul')
foul_by_team['teamId'] = ['France' if i=='Croatia' else 'Croatia' for i in foul_by_team.teamId]


foul_by_cell_start = foul.groupby(['numero_cella_x','numero_cella_y','teamId']).size().reset_index(name = 'number_foul')

foul_by_cell = foul_by_cell_start.copy()
foul_by_cell['numero_cella_x'] = np.abs(foul_by_cell['numero_cella_x'].astype(int) - 4)
foul_by_cell['numero_cella_y'] = np.abs(foul_by_cell['numero_cella_y'].astype(int) - 4)
foul_by_cell['teamId'] = ['France' if i=='Croatia' else 'Croatia' for i in foul_by_cell.teamId]

df_foul = foul_by_cell.merge(foul_by_team, on = ['teamId'])
df_foul['foul_freq'] = df_foul.number_foul/df_foul.num_total_foul

df = df.merge(df_foul[['numero_cella_x','numero_cella_y','teamId','foul_freq']], how = 'left',on = ['numero_cella_x','numero_cella_y','teamId'])

# PROBLEMA 3B 

In [60]:
df.fillna(0,inplace=True)

In [61]:
m1 =df[df.teamId == 'Croatia'].pivot_table(values='acc_pass_freq',index='numero_cella_y',columns='numero_cella_x')
m2 =df[df.teamId == 'France'].pivot_table(values='acc_pass_freq',index='numero_cella_y',columns='numero_cella_x')


def scalar_product(a,b):
    return np.sum([i*j for i,j in zip(np.ravel(a),np.ravel(b))])

def cosine_similarity(matrix1, matrix2):
    return scalar_product(matrix1, matrix2)/(np.sqrt(scalar_product(matrix1,matrix1))*np.sqrt(scalar_product(matrix2,matrix2)))

In [69]:
pd.DataFrame(a)

,0,1
0,1,0
1,0,1


In [74]:
df = df[['teamId', 'numero_cella_x', 'numero_cella_y', 'events_freq', 'acc_pass_freq', 'shot_freq', 'foul_freq']]
df = df.round({i: 2 for i in COLUMNS_NAME[3:]})
df.columns = COLUMNS_NAME

In [75]:
df

,nome_squadra,numero_cella_x,numero_cella_y,frequenza_eventi,frequenza_passaggi_accurati,frequenza_tiri,frequenza_falli_subiti
0,Croatia,0,0,0.026713,0.016317,0.000000,0.000000
1,Croatia,0,1,0.018583,0.018648,0.000000,0.000000
2,Croatia,0,2,0.013937,0.006993,0.000000,0.000000
3,Croatia,0,3,0.015099,0.020979,0.000000,0.000000
4,Croatia,0,4,0.012776,0.004662,0.000000,0.000000
5,Croatia,1,0,0.042973,0.030303,0.000000,0.000000
6,Croatia,1,1,0.054588,0.058275,0.000000,0.000000
7,Croatia,1,2,0.046458,0.053613,0.000000,0.166667
8,Croatia,1,3,0.072009,0.095571,0.000000,0.000000
9,Croatia,1,4,0.069686,0.065268,0.000000,0.083333


In [88]:
df_FR = df[df.nome_squadra == 'France']
df_CR = df[df.nome_squadra == 'Croatia']
all = df_CR.merge(df_FR, on = ['numero_cella_x','numero_cella_y'],suffixes=('_cr','_fr'))


In [93]:
ev = all[['numero_cella_x','numero_cella_y','frequenza_eventi_cr','frequenza_eventi_fr']].copy()
ev['differenza_di_frequenza'] = np.abs(ev.frequenza_eventi_cr - ev.frequenza_eventi_fr)
ev['squadra_dominante'] = ['Croatia' if (i - j) > 0 else 'Tied' if (i-j)==0 else 'France' for i,j in zip(ev.frequenza_eventi_cr,ev.frequenza_eventi_fr)]
ev.drop(['frequenza_eventi_cr','frequenza_eventi_fr'],1,inplace=True)
ev['tipo_griglia']=[1 for i in range(len(ev))]
ev

,numero_cella_x,numero_cella_y,differenza_di_frequenza,squadra_dominante,tipo_griglia
0,0,0,0.041507,France,1
1,0,1,0.008039,France,1
2,0,2,0.035980,France,1
3,0,3,0.024835,France,1
4,0,4,0.013846,France,1
5,1,0,0.040221,France,1
6,1,1,0.011968,France,1
7,1,2,0.006524,Croatia,1
8,1,3,0.017101,Croatia,1
9,1,4,0.013114,Croatia,1


,numero_cella_x,numero_cella_y,differenza_di_frequenza,squadra_dominante,tipo_griglia
0,0,0,0.041507,France,1
1,0,1,0.008039,France,1
2,0,2,0.035980,France,1
3,0,3,0.024835,France,1
4,0,4,0.013846,France,1
5,1,0,0.040221,France,1
6,1,1,0.011968,France,1
7,1,2,0.006524,Croatia,1
8,1,3,0.017101,Croatia,1
9,1,4,0.013114,Croatia,1
